In [ ]:
!pip install -q gradio
!pip install -q transformers

In [ ]:
import gradio as gr
from transformers import pipeline

# Translation Pipeline

In [ ]:
translation_pipeline = pipeline("translation_en_to_fr")

In [ ]:
result = translation_pipeline("I like mathematics and computers")

In [ ]:
result[0]["translation_text"]

In [ ]:
def translate_gradio(input):
    result = translation_pipeline(input)
    return result[0]['translation_text']

In [ ]:
translate_interface = gr.Interface(fn = translate_gradio,
                                   inputs="text",
                                   outputs="text")

In [ ]:
translate_interface.launch()

# Sentiment Analysis

In [ ]:
senti_pipeline = pipeline("sentiment-analysis")

In [ ]:
senti_pipeline("I am extremely happy to share this video with all of you")

In [ ]:
senti_pipeline("I am sad that you haven't subscribed to my channel yet")

In [ ]:
def sentiment_gradio(input):
    result = senti_pipeline(input)
    if result[0]["label"] == "POSITIVE":
        pos_score = round(result[0]["score"], 2)
        neg_score = 1 - pos_score
    else:
        neg_score = round(1*result[0]["score"], 2)
        pos_score = 1 - neg_score

    res = {'Positive': pos_score, 'Negative': neg_score}
    return res

In [ ]:
sentiment_analysis_interface = gr.Interface(fn = sentiment_gradio,
                                            inputs="text",
                                            outputs= gr.outputs.Label(num_top_classes=2),
                                            interpretation="default")

In [ ]:
sentiment_analysis_interface.launch(debug=True)

# Classification

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import logging

In [ ]:
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
df = pd.read_csv("Data.csv")

In [ ]:
df.head(3)

In [ ]:
df["is_disaster"].value_counts()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(df["Tweets"].values, 
                                                  df["is_disaster"].values, 
                                                  test_size=0.2)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(list(x_train), truncation=True, padding=True)
val_encodings = tokenizer(list(x_val), truncation=True, padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

In [ ]:
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=["accuracy"])

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=1, 
          validation_data=val_dataset.shuffle(1000).batch(16))

In [ ]:
predict_input = tokenizer.encode("thunderstorms are increasing in Mumbai",
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

In [ ]:
tf_output = model.predict(predict_input)[0]

In [ ]:
np.argmax(tf_output[0])